# Part 1 - Deeplearning for classification problem

## Import

In [6]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [7]:
tf.__version__

'2.2.0'

In [10]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [19]:
X = dataset.iloc[:, 3:-1].values
Y = dataset.iloc[:, -1].values

print (X)
print (Y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


## Categorical Data/Encoding

In [48]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X[:, 2])
print(X[:, 1])

[1 1 1 ... 1 0 1]
[0.0 0.0 0.0 ... 0.0 1.0 0.0]


## One Hot Encoding for Geography field

In [49]:
print("before", X[:, 1])
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
# print(X)
print(X[:, 1])

before [0.0 0.0 0.0 ... 0.0 1.0 0.0]
[0.0 0.0 0.0 ... 0.0 1.0 0.0]


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)


## Feature Scaling - Required always for Deeplearning

In [38]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# print(X_train)
print(X_test)

[[-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
   1.61085707]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   0.49587037]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
  -0.42478674]
 ...
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   0.71888467]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -1.54507805]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   1.61255917]]


# Part 2 - Buid the ANN

### Initialize ANN

In [39]:
ann = tf.keras.models.Sequential()

## Add input layer and first hidden layer

In [40]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Add Second hidden layer

In [41]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Add Output layer

# Part 3 - Train the ANN

## Compiling ANN

In [43]:
ann.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics =['accuracy'])

## Train ANN on training set over some epochs
 * Note: batch_size is mostly favored to be 32 by most people. batch processing has drawbacks too with large data sets
 * epochs should not be too small since aggregated info helps deeplearning fix the weights more accurately

In [44]:
ann.fit(X_train, Y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 624us/step - loss: 0.5897 - accuracy: 0.7600
Epoch 2/100
250/250 [==============================] - 0s 605us/step - loss: 0.4732 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 624us/step - loss: 0.4496 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 615us/step - loss: 0.4395 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 602us/step - loss: 0.4331 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 597us/step - loss: 0.4282 - accuracy: 0.7958
Epoch 7/100
250/250 [==============================] - 0s 620us/step - loss: 0.4237 - accuracy: 0.8104
Epoch 8/100
250/250 [==============================] - 0s 618us/step - loss: 0.4183 - accuracy: 0.8229
Epoch 9/100
250/250 [==============================] - 0s 616us/step - loss: 0.4138 - accuracy: 0.8266
Epoch 10/100
250/250 [==============================] - 0s 612us/step - l

# Part 4

## Predict the result of single observation

### New Dataset to test
Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?


In [57]:
# Predict method would only take 2D array which is [[]]
# France ==> 1.0, 0.0, 0.0 ==> 1,0,0
# We need to use one hot encoder too
# 

print (ann.predict(sc.transform([[1,0,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)


[[ True]]


Therefore, our ANN model predicts that this customer stays in the bank!

Important note 1: Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

Important note 2: Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.


In [58]:
Y_pred = (ann.predict(X_test) > 0.5)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [59]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
accuracy_score(Y_test, Y_pred)

[[1516   79]
 [ 199  206]]


0.861